In [3]:
import pandas as pd

In [21]:
df = pd.read_csv('metrobike-2020.csv')
df.head()

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type
0,134867493,25,1/1/2020 0:16,1/1/2020 0:41,3063,34.048,-118.254,4491,34.0474,-118.248,18419,30,One Way,Monthly Pass,electric
1,134867799,35,1/1/2020 0:24,1/1/2020 0:59,4285,NaN,NaN,4354,34.0177,-118.409,15661,1,One Way,One Day Pass,smart
2,134868104,37,1/1/2020 0:31,1/1/2020 1:08,4344,34.0143,-118.491,4322,34.0059,-118.429,15848,1,One Way,Walk-up,smart
3,134868103,36,1/1/2020 0:32,1/1/2020 1:08,4344,34.0143,-118.491,4322,34.0059,-118.429,16053,1,One Way,Walk-up,smart
4,134868102,35,1/1/2020 0:33,1/1/2020 1:08,4344,34.0143,-118.491,4322,34.0059,-118.429,15694,1,One Way,Walk-up,smart


In [25]:
df.columns

Index(['trip_id', 'duration', 'start_time', 'end_time', 'start_station',
       'start_lat', 'start_lon', 'end_station', 'end_lat', 'end_lon',
       'bike_id', 'plan_duration', 'trip_route_category', 'passholder_type',
       'bike_type'],
      dtype='object')

In [37]:
# Calculate number of rides by bicycle type 
a = df['bike_type'].value_counts()
# a.value_counts(normalize=True)
a
# Calculate number of unique bicycles by type 


standard     98580
electric     82207
smart        29187
bike_type        3
Name: bike_type, dtype: int64